In [139]:
import nltk
from nltk.corpus import brown
from nltk.tokenize import sent_tokenize
import string
from collections import defaultdict
from collections import Counter
import numpy as np
import math

In [34]:
corp = brown.sents(categories='news')
brown_news_tagged = brown.tagged_sents(categories='news', tagset='universal')

In [134]:
flat_corp = [item for l in corp for item in l]

In [74]:
flat_brown = [item for l in brown_news_tagged for item in l]

In [137]:
pi_DET = [1]

In [77]:
b = Counter(flat_brown)

In [101]:
b_ik = defaultdict(dict)
for w_pos, c in b.items():
    b_ik[w_pos[0]][w_pos[1]] = c

In [121]:
denom = defaultdict(int)
for w_pos, c in b.items():
    denom[w_pos[1]] += c

In [122]:
for w, d in b_ik.items():
    for pos, c in d.items():
        d[pos] = c * 1.0 / denom[pos]

In [123]:
a_ij = defaultdict(dict)

for c in range(0, len(flat_brown) - 1):
    if flat_brown[c+1][1] in a_ij[flat_brown[c][1]]:
        a_ij[flat_brown[c][1]][flat_brown[c+1][1]] += 1
    else:
        a_ij[flat_brown[c][1]][flat_brown[c+1][1]] = 1

In [124]:
for pos, d in a_ij.items():
    for p, c in d.items():
        d[p] = c * 1.0 / denom[pos]

In [125]:
for pos, d in a_ij.items():
    count = 0
    for p, c in d.items():
        count += c
    print(pos + ": "+ str(count))

DET: 1.0000000000000002
PRON: 1.0
ADJ: 1.0
ADP: 1.0
PRT: 1.0
NOUN: 1.0
ADV: 1.0
.: 0.999916163648558
NUM: 1.0
X: 1.0
CONJ: 1.0
VERB: 1.0


In [126]:
o_index = dict(zip(b_ik, range(len(b_ik))))

In [127]:
s_index = dict(zip(a_ij, range(len(a_ij))))

In [128]:
em = np.zeros((len(s_index), len(o_index)))
tm = np.zeros((len(s_index), len(s_index)))

In [129]:
for o, s in b_ik.items():
    for pos, prob in s.items():
        em[s_index[pos]][o_index[o]] = prob

In [130]:
for si, pos in a_ij.items():
    for sj, prob in pos. items():
        tm[s_index[si]][s_index[sj]] = prob

In [151]:
def helper(col, tm):
    mx = (-1, -math.inf)

    for row in range(len(col)):
        value = col[row]
        try:
            value += math.log(tm[row])
        except:
            value = 0

        if value > mx[1]:
            mx = (row, value)

    return mx

In [156]:
def run(em, tm, obs, init):
    n = len(em); T = len(obs)
    # step 1: init n x T matrix
    ls = np.zeros((n, T))
    phi = np.zeros((n, T))


    # col 1 in ls -> log(pi_i*bi_o1)
    for r in range(len(init)):
        value = init[r]*em[r][o_index[obs[0]]]
        if (value == 0):
            ls[r][0] = 0
        else:
            ls[r][0] = math.log(value)


    #step 2: fill in tables by col
    for t in range(1,T): # t -> col (time)

        for s in range(0,n): # s -> row (state)
            try:
                prob_event = math.log( em[s][o_index[obs[t]]] ) # log( bj(Ot+1))
            except: 
                prob_event = 0

            mx = helper(ls[:, t-1], tm[s][:])

            value = prob_event + mx[1]
            ls[s][t] = value
            phi[s][t] = mx[0]

    last_col = ls[:, np.shape(ls)[1]-1]

    start = np.argmax(last_col)
    path = [start]

    for time in range(T-1, 0, -1):
        start = phi[int(start)][time]
        path = np.insert(path, 0, int(start))
    
    return path

In [144]:
init = np.zeros(len(s_index))
init[s_index['DET']] = 1

In [157]:
path = run(em, tm, flat_corp, init)

In [158]:
print(path)

[10 10 10 ...,  0  9  1]


In [159]:
s_index

{'.': 7,
 'ADJ': 2,
 'ADP': 3,
 'ADV': 6,
 'CONJ': 10,
 'DET': 0,
 'NOUN': 5,
 'NUM': 8,
 'PRON': 1,
 'PRT': 4,
 'VERB': 11,
 'X': 9}

In [30]:
if __name__ == "__main__":
    main()

['The', 'Fulton', 'County', 'Grand', 'Jury', 'said', 'Friday', 'an', 'investigation', 'of', "Atlanta's", 'recent', 'primary', 'election', 'produced', '``', 'no', 'evidence', "''", 'that', 'any', 'irregularities', 'took', 'place', '.']
['The', 'jury', 'further', 'said', 'in', 'term-end', 'presentments', 'that', 'the', 'City', 'Executive', 'Committee', ',', 'which', 'had', 'over-all', 'charge', 'of', 'the', 'election', ',', '``', 'deserves', 'the', 'praise', 'and', 'thanks', 'of', 'the', 'City', 'of', 'Atlanta', "''", 'for', 'the', 'manner', 'in', 'which', 'the', 'election', 'was', 'conducted', '.']
4623
defaultdict(<class 'int'>, {'DET': 1116, 'PRON': 435, 'ADJ': 208, 'ADP': 456, 'PRT': 127, 'NOUN': 1208, 'ADV': 251, '.': 405, 'NUM': 77, 'X': 2, 'CONJ': 168, 'VERB': 170})
4623


In [29]:
def main():
    corp = brown.sents(categories='news')
    print(corp[0])
    print(corp[1])

    brown_news_tagged = brown.tagged_sents(categories='news', tagset='universal')

    pi_s, count = pi(brown_news_tagged)
    print(pi_s)
    print(count)

In [35]:
def pi(sents):
    pos = defaultdict(int)
    count = 0
    for s in sents:
        pos[s[0][1]] += 1
        count += 1
    return pos, count